In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# prefix path
prefix = '/content/drive/MyDrive/데이터크리에이터캠프/CV/task1/jinwoo'

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [4]:
# tup
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 
    # default distribution strategy in Tensorflow. Works on CPU and single GPU.
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.31.138.26:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.31.138.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.31.138.26:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [5]:
# fashion dataset load
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
# dataset shape
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [7]:
# the number of classes
num_class = len(np.unique(test_labels))

In [8]:
# make one-hot label
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

In [9]:
# preprocess function
# normalize x
def preprocess(x, y):
  x = tf.cast(x, dtype=tf.float32)
  x = x/255.0
  x = tf.expand_dims(x, 3)
  return x, y

In [10]:
# make trainset
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.map(preprocess).shuffle(1000, seed=42)    # shuffle

test_dataset  = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.map(preprocess)

In [11]:
with strategy.scope():
  # model
  model = keras.models.Sequential([
              keras.layers.Conv2D(8, 5, activation='relu', input_shape=(28, 28, 1)),
              keras.layers.MaxPool2D(pool_size=(3, 3)),
              keras.layers.Conv2D(16, 3, activation='relu'),
              keras.layers.MaxPool2D(pool_size=(2, 2)),
              keras.layers.Flatten(),
              keras.layers.Dense(20, activation='relu'),
              keras.layers.Dense(num_class, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics='accuracy'
  )

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 8)         208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 16)          1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 144)               0         
_________________________________________________________________
dense (Dense)                (None, 20)                2900      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2

In [14]:
# callback method

# checkpoint path
ckpt_name = 'CNN_ckpt.hdf5'
checkpoint_dir_path = os.path.join(prefix, 'checkpoint')
checkpoint_path = os.path.join(prefix, 'checkpoint', ckpt_name)

# check checkpoint paht
# if not exist, make dir
if not(os.path.exists(checkpoint_dir_path)):
  os.mkdir(checkpoint_dir_path)

callbacks_list = [
    # save model weights, at every epoch
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # EarlyStopping, if val_loss is not improve five times
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=5
    )
]

In [15]:
hist = model.fit(train_dataset, validation_data=test_dataset, callbacks=callbacks_list, epochs=20)

Epoch 1/20
938/938 [==============================] - 19s 16ms/step - loss: 0.8152 - accuracy: 0.7020 - val_loss: 0.5820 - val_accuracy: 0.7808
Epoch 2/20
938/938 [==============================] - 12s 12ms/step - loss: 0.5329 - accuracy: 0.8030 - val_loss: 0.5209 - val_accuracy: 0.8064
Epoch 3/20
938/938 [==============================] - 13s 13ms/step - loss: 0.4765 - accuracy: 0.8275 - val_loss: 0.4967 - val_accuracy: 0.8217
Epoch 4/20
938/938 [==============================] - 12s 13ms/step - loss: 0.4432 - accuracy: 0.8406 - val_loss: 0.4442 - val_accuracy: 0.8405
Epoch 5/20
938/938 [==============================] - 12s 13ms/step - loss: 0.4178 - accuracy: 0.8510 - val_loss: 0.4235 - val_accuracy: 0.8517
Epoch 6/20
938/938 [==============================] - 12s 13ms/step - loss: 0.3982 - accuracy: 0.8558 - val_loss: 0.4079 - val_accuracy: 0.8565
Epoch 7/20
938/938 [==============================] - 11s 12ms/step - loss: 0.3825 - accuracy: 0.8623 - val_loss: 0.4003 - val_accuracy:

In [16]:
# load last ckpt weight
model.load_weights(checkpoint_path)